# Build an Agent

By themselves, language models can't take actions. They just output text.

A big use case for LangChain is creating **agents**.

Agents are systems that use LLMs as **reasoning engines** to **determine which actions to take and the inputs to pass them**.

After executing actions, the results can be fed back into the LLM to determine wether more actions are needed, or whether it is ok to finish.

In this tutorial, we will build **an agent that can interact with a search engine**.

You will be able to ask this agent questions, watch it call the search tool, and have conversations with it.

# Concepts

In the following tutorial, you will learn how to:
- Use [**language models**](https://python.langchain.com/v0.2/docs/concepts/#chat-models), in particular their **tool calling** ability.
- Use a Search [**Tool**](https://python.langchain.com/v0.2/docs/concepts/#tools) to look up information from the Internet.
- Compose a [**LangGraph Agent**](https://python.langchain.com/v0.2/docs/concepts/#agents), whch use an LLM to determine actions and then execute them.
- Debug and trace your application using [**LangSmith**](https://python.langchain.com/v0.2/docs/concepts/#langsmith)

# End-to-end Agent

The code snippet below represents a fully functional agent that uses an LLM to decide which tools to use.

It is equipped with a generic search tool.

It has a conversational memory, meaning it can be used as a multi-turn chatbot.

In the rest of the guide, we will walk through the individual components and what each part does.

# Setup

In [1]:
from dotenv import load_dotenv

In [2]:
_ = load_dotenv()

In [4]:
# Improve pretty printing for display purposes
from rich import print as rprint

# Define Tools

We first need to create the tools we want to use.

Our main tool of choice will be [**Tavily**](https://python.langchain.com/v0.2/docs/integrations/tools/tavily_search/), a search engine. We have a built-in tool in LangChain to easily use Tavily search engine as a tool.

In [3]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [6]:
search = TavilySearchResults(max_results=2)

In [7]:
search_results = search.invoke("What is the weather in SF?")

In [9]:
rprint(search_results)

[
    {
        'url': 'https://www.weatherapi.com/',
        'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of 
America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1718465317, 'localtime':
'2024-06-15 8:28'}, 'current': {'last_updated_epoch': 1718464500, 'last_updated': '2024-06-15 08:15', 'temp_c': 
15.3, 'temp_f': 59.5, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': 
'//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 3.8, 'wind_kph': 6.1, 'wind_degree': 
180, 'wind_dir': 'S', 'pressure_mb': 1017.0, 'pressure_in': 30.04, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 
69, 'cloud': 25, 'feelslike_c': 15.3, 'feelslike_f': 59.5, 'windchill_c': 12.9, 'windchill_f': 55.2, 'heatindex_c':
12.8, 'heatindex_f': 55.0, 'dewpoint_c': 8.5, 'dewpoint_f': 47.3, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 4.0, 
'gust_mph': 4.1, 'gust_kph': 6.6}}"
    },
    {
        'url': 'https://world-weather.info/forecast/usa/san_francisco/june-2024/',
        'content': 'Extended weather forecast in San Francisco. Hourly Week 10 days 14 days 30 days Year. Detailed 
⚡ San Francisco Weather Forecast for June 2024 - day/night 🌡️ temperatures, precipitations - World-Weather.info.'
    }
]

In [12]:
# Define tools
tools = [search]

# Using Language Models

Now, let's learn how to use a language model to call tools.

LangChain supports many different language models that you can use interchangably.

In [10]:
from langchain_openai import ChatOpenAI

In [11]:
model = ChatOpenAI(model="gpt-3.5-turbo")

In [13]:
rprint(model)

ChatOpenAI(
    client=<openai.resources.chat.completions.Completions object at 0x7e5769bfc550>,
    async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7e5769c0f2d0>,
    openai_api_key=SecretStr('**********'),
    openai_proxy=''
)

To enable this model to perform **tool calling**, we will use the `.bind_tools` method, to give the LLM knowledge of these tools.

In [16]:
model_with_tools = model.bind_tools(tools)

In [17]:
rprint(model_with_tools)

RunnableBinding(
    bound=ChatOpenAI(
        client=<openai.resources.chat.completions.Completions object at 0x7e5769bfc550>,
        async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7e5769c0f2d0>,
        openai_api_key=SecretStr('**********'),
        openai_proxy=''
    ),
    kwargs={
        'tools': [
            {
                'type': 'function',
                'function': {
                    'name': 'tavily_search_results_json',
                    'description': 'A search engine optimized for comprehensive, accurate, and trusted results. 
Useful for when you need to answer questions about current events. Input should be a search query.',
                    'parameters': {
                        'type': 'object',
                        'properties': {'query': {'description': 'search query to look up', 'type': 'string'}},
                        'required': ['query']
                    }
                }
            }
        ]
    }
)

In [19]:
# Focus
rprint(model_with_tools.kwargs["tools"])

[
    {
        'type': 'function',
        'function': {
            'name': 'tavily_search_results_json',
            'description': 'A search engine optimized for comprehensive, accurate, and trusted results. Useful for 
when you need to answer questions about current events. Input should be a search query.',
            'parameters': {
                'type': 'object',
                'properties': {'query': {'description': 'search query to look up', 'type': 'string'}},
                'required': ['query']
            }
        }
    }
]

We can now call this model.

Let's first call it with a simple message, not requiring any Web search, and see how it responds.

For introspection purposes, we will, as usually, inspect the `content` attribute, but the `tool_calls` one too.

In [20]:
from langchain_core.messages import HumanMessage

In [21]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

In [22]:
rprint(response.content)

Hello! How can I assist you today?

In [24]:
rprint(response.tool_calls)

[]

This confirms that the tool hasn't been called for this simple message.

Now, let's try calling it with some input that would expect a tool to be called.

In [38]:
response = model_with_tools.invoke([HumanMessage(content="When will the next NBA finals match be played?")])

In [39]:
response.content

''

In [40]:
response.tool_calls

[{'name': 'tavily_search_results_json',
  'args': {'query': 'NBA finals next match date'},
  'id': 'call_UZ4UOibIiWS4ovFwVJryoxxa'}]

You can see that
- surprisingly, **there's not text content**,
- **there is a tool call**.

In fact...
- this isn't calling the tool now,
- ut's telling us that it has *chosen* to do so.

# Create the Agent

Now that we have defined the tools and the LLM, we can create the agent.

We will be using [**LangGraph**](https://python.langchain.com/v0.2/docs/concepts/#langgraph) to construct it.

Currently, we are using a high-level interface to construct the agent, but the nice thing about LangGraph is that this high-level interface is backed-up by a low-level, highly controllable API in case you want to modify the agent logic.

Now, we can initialize the agent with:
- the LLM,
- the tools.

In [41]:
from langgraph.prebuilt import create_react_agent

In [61]:
agent_executor = create_react_agent(model, tools)

> **NOTE**
> 
> Note that, right now, we just passed `model` and not `model_with_tools`. This is because `create_react_agent` will call `.bind_tools` under the hood for us.

## Introspection

In [70]:
type(agent_executor)

langgraph.graph.state.CompiledStateGraph

### `__str__`

In [63]:
rprint(agent_executor)

CompiledStateGraph(
    nodes={
        '__start__': PregelNode(
            config={'tags': ['langsmith:hidden']},
            channels=['__start__'],
            triggers=['__start__'],
            writers=[
                ChannelWrite<messages>(recurse=True, writes=[ChannelWriteEntry(channel='messages', value=<object 
object at 0x7e5768f5cc30>, skip_none=False, mapper=_get_state_key(recurse=False))], 
require_at_least_one_of=['messages']),
                ChannelWrite<start:agent>(recurse=True, writes=[ChannelWriteEntry(channel='start:agent', 
value='__start__', skip_none=False, mapper=None)], require_at_least_one_of=None)
            ]
        ),
        'agent': PregelNode(
            config={'tags': []},
            channels={
                'messages': 'messages',
                'is_last_step': <class 'langgraph.managed.is_last_step.IsLastStepManager'>
            },
            triggers=['start:agent', 'tools'],
            mapper=functools.partial(<function _coerce_state at 0x7e5768db4e00>, <class 
'langgraph.prebuilt.chat_agent_executor.AgentState'>),
            writers=[
                ChannelWrite<agent,messages>(recurse=True, writes=[ChannelWriteEntry(channel='agent', 
value='agent', skip_none=False, mapper=None), ChannelWriteEntry(channel='messages', value=<object object at 
0x7e5768f5cc30>, skip_none=False, mapper=_get_state_key(recurse=False))], require_at_least_one_of=['messages']),
                _route(recurse=True, _is_channel_writer=True)
            ]
        ),
        'tools': PregelNode(
            config={'tags': []},
            channels={
                'messages': 'messages',
                'is_last_step': <class 'langgraph.managed.is_last_step.IsLastStepManager'>
            },
            triggers=['branch:agent:should_continue:tools'],
            mapper=functools.partial(<function _coerce_state at 0x7e5768db4e00>, <class 
'langgraph.prebuilt.chat_agent_executor.AgentState'>),
            writers=[
                ChannelWrite<tools,messages>(recurse=True, writes=[ChannelWriteEntry(channel='tools', 
value='tools', skip_none=False, mapper=None), ChannelWriteEntry(channel='messages', value=<object object at 
0x7e5768f5cc30>, skip_none=False, mapper=_get_state_key(recurse=False))], require_at_least_one_of=['messages'])
            ]
        )
    },
    channels={
        'messages': <langgraph.channels.binop.BinaryOperatorAggregate object at 0x7e574ae06ad0>,
        '__start__': <langgraph.channels.ephemeral_value.EphemeralValue object at 0x7e574ad01510>,
        'agent': <langgraph.channels.ephemeral_value.EphemeralValue object at 0x7e574ad03550>,
        'tools': <langgraph.channels.ephemeral_value.EphemeralValue object at 0x7e574ad00a90>,
        'start:agent': <langgraph.channels.ephemeral_value.EphemeralValue object at 0x7e574aee7f90>,
        'branch:agent:should_continue:tools': <langgraph.channels.ephemeral_value.EphemeralValue object at 
0x7e574ad649d0>
    },
    auto_validate=False,
    stream_mode='updates',
    output_channels=['messages'],
    stream_channels=['messages'],
    input_channels='__start__',
    builder=<langgraph.graph.state.StateGraph object at 0x7e5769c0b710>
)

### Attributes

In [66]:
agent_attrs = [item for item in dir(agent_executor) if item[0] != "_"]
len(agent_attrs)

84

In [68]:
# agent_attrs

### Exploring `__str__`

In [73]:
type(agent_executor.nodes)

dict

In [74]:
agent_executor.nodes.keys()

dict_keys(['__start__', 'agent', 'tools'])

In [75]:
rprint(agent_executor.nodes["__start__"])

PregelNode(
    config={'tags': ['langsmith:hidden']},
    channels=['__start__'],
    triggers=['__start__'],
    writers=[
        ChannelWrite<messages>(recurse=True, writes=[ChannelWriteEntry(channel='messages', value=<object object at 
0x7e5768f5cc30>, skip_none=False, mapper=_get_state_key(recurse=False))], require_at_least_one_of=['messages']),
        ChannelWrite<start:agent>(recurse=True, writes=[ChannelWriteEntry(channel='start:agent', value='__start__',
skip_none=False, mapper=None)], require_at_least_one_of=None)
    ]
)

In [76]:
rprint(agent_executor.nodes["agent"])

PregelNode(
    config={'tags': []},
    channels={'messages': 'messages', 'is_last_step': <class 'langgraph.managed.is_last_step.IsLastStepManager'>},
    triggers=['start:agent', 'tools'],
    mapper=functools.partial(<function _coerce_state at 0x7e5768db4e00>, <class 
'langgraph.prebuilt.chat_agent_executor.AgentState'>),
    writers=[
        ChannelWrite<agent,messages>(recurse=True, writes=[ChannelWriteEntry(channel='agent', value='agent', 
skip_none=False, mapper=None), ChannelWriteEntry(channel='messages', value=<object object at 0x7e5768f5cc30>, 
skip_none=False, mapper=_get_state_key(recurse=False))], require_at_least_one_of=['messages']),
        _route(recurse=True, _is_channel_writer=True)
    ]
)

In [77]:
rprint(agent_executor.nodes["tools"])

PregelNode(
    config={'tags': []},
    channels={'messages': 'messages', 'is_last_step': <class 'langgraph.managed.is_last_step.IsLastStepManager'>},
    triggers=['branch:agent:should_continue:tools'],
    mapper=functools.partial(<function _coerce_state at 0x7e5768db4e00>, <class 
'langgraph.prebuilt.chat_agent_executor.AgentState'>),
    writers=[
        ChannelWrite<tools,messages>(recurse=True, writes=[ChannelWriteEntry(channel='tools', value='tools', 
skip_none=False, mapper=None), ChannelWriteEntry(channel='messages', value=<object object at 0x7e5768f5cc30>, 
skip_none=False, mapper=_get_state_key(recurse=False))], require_at_least_one_of=['messages'])
    ]
)

# Run the Agent

We can now run the agent on a few queries!

Note that, for now, these are all **stateless** queries (it won't remember previous interactions).

Note also that the agent will return the **final** state, at the end of the interaction (which includes any inputs. We will see later how to get only the outputs).

## With No Need of Tool Call

First up, let's see how it responds when there's no need to perform a tool call

In [44]:
response = agent_executor.invoke({"messages": [HumanMessage(content="Hi!")]})

In [45]:
rprint(response["messages"])

[
    HumanMessage(content='Hi!', id='4a64ae5d-3002-48c1-a33c-68431ba79ce3'),
    AIMessage(
        content='Hello! How can I assist you today?',
        response_metadata={
            'token_usage': {'completion_tokens': 10, 'prompt_tokens': 83, 'total_tokens': 93},
            'model_name': 'gpt-3.5-turbo',
            'system_fingerprint': None,
            'finish_reason': 'stop',
            'logprobs': None
        },
        id='run-c1904b7e-b3de-403f-82ae-62e44b3a6e67-0',
        usage_metadata={'input_tokens': 83, 'output_tokens': 10, 'total_tokens': 93}
    )
]

In order to see exactly what is happening under the hood (and to make sure it's not calling a tool) we can take a look at the [**LangSmith trace**](https://smith.langchain.com/public/28311faa-e135-4d6a-ab6b-caecf6482aaa/r)

In [47]:
# INTROSPECTION
type(response)

langgraph.pregel.io.AddableValuesDict

In [49]:
# INTROSPECTION: inspecting the whole object
rprint(response)

{
    'messages': [
        HumanMessage(content='Hi!', id='4a64ae5d-3002-48c1-a33c-68431ba79ce3'),
        AIMessage(
            content='Hello! How can I assist you today?',
            response_metadata={
                'token_usage': {'completion_tokens': 10, 'prompt_tokens': 83, 'total_tokens': 93},
                'model_name': 'gpt-3.5-turbo',
                'system_fingerprint': None,
                'finish_reason': 'stop',
                'logprobs': None
            },
            id='run-c1904b7e-b3de-403f-82ae-62e44b3a6e67-0',
            usage_metadata={'input_tokens': 83, 'output_tokens': 10, 'total_tokens': 93}
        )
    ]
}

> **NOTE**
> 
> For the moment, it appears as a `dict` with only the `"messages"` key

## With the Need of Tool Call

In [51]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in Lille?")]}
)

In [52]:
rprint(response["messages"])

[
    HumanMessage(content='whats the weather in Lille?', id='d185f7ce-c8ee-4b16-9063-b887797f2f54'),
    AIMessage(
        content='',
        additional_kwargs={
            'tool_calls': [
                {
                    'id': 'call_uAEbyA6mFg5rciGwNzUZZECQ',
                    'function': {
                        'arguments': '{"query":"weather in Lille"}',
                        'name': 'tavily_search_results_json'
                    },
                    'type': 'function'
                }
            ]
        },
        response_metadata={
            'token_usage': {'completion_tokens': 21, 'prompt_tokens': 89, 'total_tokens': 110},
            'model_name': 'gpt-3.5-turbo',
            'system_fingerprint': None,
            'finish_reason': 'tool_calls',
            'logprobs': None
        },
        id='run-7f006750-0273-4d67-8d44-139fd7721745-0',
        tool_calls=[
            {
                'name': 'tavily_search_results_json',
                'args': {'query': 'weather in Lille'},
                'id': 'call_uAEbyA6mFg5rciGwNzUZZECQ'
            }
        ],
        usage_metadata={'input_tokens': 89, 'output_tokens': 21, 'total_tokens': 110}
    ),
    ToolMessage(
        content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'Lille\', 
\'region\': \'Nord-Pas-de-Calais\', \'country\': \'France\', \'lat\': 50.63, \'lon\': 3.07, \'tz_id\': 
\'Europe/Paris\', \'localtime_epoch\': 1718469574, \'localtime\': \'2024-06-15 18:39\'}, \'current\': 
{\'last_updated_epoch\': 1718469000, \'last_updated\': \'2024-06-15 18:30\', \'temp_c\': 15.1, \'temp_f\': 59.2, 
\'is_day\': 1, \'condition\': {\'text\': \'Moderate or heavy rain with thunder\', \'icon\': 
\'//cdn.weatherapi.com/weather/64x64/day/389.png\', \'code\': 1276}, \'wind_mph\': 21.7, \'wind_kph\': 34.9, 
\'wind_degree\': 200, \'wind_dir\': \'SSW\', \'pressure_mb\': 1005.0, \'pressure_in\': 29.68, \'precip_mm\': 0.08, 
\'precip_in\': 0.0, \'humidity\': 77, \'cloud\': 75, \'feelslike_c\': 15.1, \'feelslike_f\': 59.2, \'windchill_c\':
11.7, \'windchill_f\': 53.0, \'heatindex_c\': 13.6, \'heatindex_f\': 56.5, \'dewpoint_c\': 11.4, \'dewpoint_f\': 
52.4, \'vis_km\': 10.0, \'vis_miles\': 6.0, \'uv\': 3.0, \'gust_mph\': 22.4, \'gust_kph\': 36.0}}"}, {"url": 
"https://weatherspark.com/h/y/49799/2024/Historical-Weather-during-2024-in-Lille-France", "content": "2024 Weather 
History in Lille France. The data for this report comes from the Lille Airport. ... LFQQ 051800Z AUTO 28010KT CAVOK
18/06 Q1015 NOSIG. This report shows the past weather for Lille, providing a weather history for 2024. ... frigid 
15\\u00b0F freezing 32\\u00b0F very cold 45\\u00b0F cold 55\\u00b0F cool 65\\u00b0F comfortable 75\\u00b0F warm 
85\\u00b0F hot 95\\u00b0F ..."}]',
        name='tavily_search_results_json',
        id='98c19b69-82d2-471b-8d02-2768fe3901b0',
        tool_call_id='call_uAEbyA6mFg5rciGwNzUZZECQ'
    ),
    AIMessage(
        content='The current weather in Lille, France is as follows:\n- Temperature: 15.1°C (59.2°F)\n- Condition: 
Moderate or heavy rain with thunder\n- Wind: 34.9 km/h from SSW\n- Pressure: 1005.0 mb\n- Humidity: 77%\n- Cloud 
Cover: 75%\n\nFor more detailed information, you can visit [Weather API](https://www.weatherapi.com/).',
        response_metadata={
            'token_usage': {'completion_tokens': 94, 'prompt_tokens': 708, 'total_tokens': 802},
            'model_name': 'gpt-3.5-turbo',
            'system_fingerprint': None,
            'finish_reason': 'stop',
            'logprobs': None
        },
        id='run-2e13b72f-f5a7-42a8-89bb-b046c1509b8d-0',
        usage_metadata={'input_tokens': 708, 'output_tokens': 94, 'total_tokens': 802}
    )
]

We can check out the [**LangSmith trace**](https://smith.langchain.com/public/f520839d-cd4d-4495-8764-e32b548e235d/r) to make sure it's calling the search tool effectively.

# Streaming Messages

We've seen how the agent can be called with `.invoke` to get back a final response.

However, if the agent is executing multiple steps, that may take a while.

In order to show intermediate progress, we can stream back messages as they occur.

In [54]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in Lille?")]}
):
    rprint(chunk)
    rprint("-" * 100)

{
    'agent': {
        'messages': [
            AIMessage(
                content='',
                additional_kwargs={
                    'tool_calls': [
                        {
                            'id': 'call_98w0gO1aCFLraoTM4EMIaxEY',
                            'function': {
                                'arguments': '{"query":"weather in Lille"}',
                                'name': 'tavily_search_results_json'
                            },
                            'type': 'function'
                        }
                    ]
                },
                response_metadata={
                    'token_usage': {'completion_tokens': 21, 'prompt_tokens': 89, 'total_tokens': 110},
                    'model_name': 'gpt-3.5-turbo',
                    'system_fingerprint': None,
                    'finish_reason': 'tool_calls',
                    'logprobs': None
                },
                id='run-008a07db-7a61-4cd9-b67c-9feabb65ba0c-0',
                tool_calls=[
                    {
                        'name': 'tavily_search_results_json',
                        'args': {'query': 'weather in Lille'},
                        'id': 'call_98w0gO1aCFLraoTM4EMIaxEY'
                    }
                ],
                usage_metadata={'input_tokens': 89, 'output_tokens': 21, 'total_tokens': 110}
            )
        ]
    }
}

----------------------------------------------------------------------------------------------------

{
    'tools': {
        'messages': [
            ToolMessage(
                content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'Lille\', 
\'region\': \'Nord-Pas-de-Calais\', \'country\': \'France\', \'lat\': 50.63, \'lon\': 3.07, \'tz_id\': 
\'Europe/Paris\', \'localtime_epoch\': 1718469962, \'localtime\': \'2024-06-15 18:46\'}, \'current\': 
{\'last_updated_epoch\': 1718469900, \'last_updated\': \'2024-06-15 18:45\', \'temp_c\': 15.3, \'temp_f\': 59.5, 
\'is_day\': 1, \'condition\': {\'text\': \'Moderate or heavy rain with thunder\', \'icon\': 
\'//cdn.weatherapi.com/weather/64x64/day/389.png\', \'code\': 1276}, \'wind_mph\': 19.2, \'wind_kph\': 31.0, 
\'wind_degree\': 200, \'wind_dir\': \'SSW\', \'pressure_mb\': 1005.0, \'pressure_in\': 29.68, \'precip_mm\': 0.08, 
\'precip_in\': 0.0, \'humidity\': 77, \'cloud\': 75, \'feelslike_c\': 15.3, \'feelslike_f\': 59.5, \'windchill_c\':
11.7, \'windchill_f\': 53.0, \'heatindex_c\': 13.6, \'heatindex_f\': 56.5, \'dewpoint_c\': 11.4, \'dewpoint_f\': 
52.4, \'vis_km\': 10.0, \'vis_miles\': 6.0, \'uv\': 3.0, \'gust_mph\': 22.4, \'gust_kph\': 36.0}}"}, {"url": 
"https://www.accuweather.com/en/fr/lille/135564/june-weather/135564", "content": "Get the monthly weather forecast 
for Lille, Nord, France, including daily high/low, historical averages, to help you plan ahead."}]',
                name='tavily_search_results_json',
                tool_call_id='call_98w0gO1aCFLraoTM4EMIaxEY'
            )
        ]
    }
}

----------------------------------------------------------------------------------------------------

{
    'agent': {
        'messages': [
            AIMessage(
                content='The current weather in Lille, France is as follows:\n- Temperature: 15.3°C (59.5°F)\n- 
Condition: Moderate or heavy rain with thunder\n- Wind: 31.0 km/h from SSW\n- Humidity: 77%\n- Cloud Cover: 75%\n- 
Visibility: 10.0 km\n- UV Index: 3.0\n\nFor more detailed weather information and forecasts, you can visit [Weather
API](https://www.weatherapi.com/).\n\nIf you need to plan ahead and get the monthly weather forecast for Lille, you
can check [AccuWeather](https://www.accuweather.com/en/fr/lille/135564/june-weather/135564).',
                response_metadata={
                    'token_usage': {'completion_tokens': 151, 'prompt_tokens': 594, 'total_tokens': 745},
                    'model_name': 'gpt-3.5-turbo',
                    'system_fingerprint': None,
                    'finish_reason': 'stop',
                    'logprobs': None
                },
                id='run-8b11ae5b-ae72-41ec-b92d-9c7249f945eb-0',
                usage_metadata={'input_tokens': 594, 'output_tokens': 151, 'total_tokens': 745}
            )
        ]
    }
}

----------------------------------------------------------------------------------------------------

# Streaming Tokens

In addition to streaming back messages, it is also useful to be streaming back tokens, which can be done with the `.astrema_events` method.

> **NOTE**: This only works with Python 3.11 or higher.

In [55]:
async for event in agent_executor.astream_events(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}, version="v1"
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

--
Starting tool: tavily_search_results_json with inputs: {'query': 'weather in San Francisco'}
Done tool: tavily_search_results_json
Tool output was: [{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1718470129, 'localtime': '2024-06-15 9:48'}, 'current': {'last_updated_epoch': 1718469900, 'last_updated': '2024-06-15 09:45', 'temp_c': 13.9, 'temp_f': 57.0, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 2.9, 'wind_kph': 4.7, 'wind_degree': 266, 'wind_dir': 'W', 'pressure_mb': 1017.0, 'pressure_in': 30.03, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 69, 'cloud': 0, 'feelslike_c': 13.8, 'feelslike_f': 56.9, 'windchill_c': 13.8, 'windchill_f': 56.9, 'heatindex_c': 13.9, 'heatindex_f': 57.0, 'dewpoint_c': 8.2, 'dewpoint_f':

# Adding in Memory

As mentioned earlier, **this agent is stateless**. This means **it doesn't remember previous interactions**.

To give it memory, we need to pass in a **checkpointer**.

When passing in a checkpointer, **we also have to pass in a `thread_id` when invoking the agent** (so it knows which thread/conversation to resume from.)

In [82]:
from langgraph.checkpoint.sqlite import SqliteSaver

In [83]:
# Instanciate an in memory checkpointer
memory = SqliteSaver.from_conn_string(":memory:")

> **DOCUMENTATION**: 
- [**Checkpoints**](https://langchain-ai.github.io/langgraph/reference/checkpoints/)
- [**SqliteSaver**](https://langchain-ai.github.io/langgraph/reference/checkpoints/#sqlitesaver)

In [84]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [85]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Hi! I'm Bob.")]},
    config
):
    rprint(chunk)
    print("-" * 100)

{
    'agent': {
        'messages': [
            AIMessage(
                content='Hello Bob! How can I assist you today?',
                response_metadata={
                    'token_usage': {'completion_tokens': 11, 'prompt_tokens': 87, 'total_tokens': 98},
                    'model_name': 'gpt-3.5-turbo',
                    'system_fingerprint': None,
                    'finish_reason': 'stop',
                    'logprobs': None
                },
                id='run-8c3522cb-f0cb-4b29-bd4b-8180c1c2e9c4-0',
                usage_metadata={'input_tokens': 87, 'output_tokens': 11, 'total_tokens': 98}
            )
        ]
    }
}

----------------------------------------------------------------------------------------------------


In [86]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="What's my name?")]},
    config
):
    rprint(chunk)
    print("-" * 100)

{
    'agent': {
        'messages': [
            AIMessage(
                content='Your name is Bob. How can I help you, Bob?',
                response_metadata={
                    'token_usage': {'completion_tokens': 14, 'prompt_tokens': 110, 'total_tokens': 124},
                    'model_name': 'gpt-3.5-turbo',
                    'system_fingerprint': None,
                    'finish_reason': 'stop',
                    'logprobs': None
                },
                id='run-1fcabe0c-291c-4152-9613-4ff42a46469b-0',
                usage_metadata={'input_tokens': 110, 'output_tokens': 14, 'total_tokens': 124}
            )
        ]
    }
}

----------------------------------------------------------------------------------------------------


Example [**LangSmith trace**](https://smith.langchain.com/public/fa73960b-0f7d-4910-b73d-757a12f33b2b/r)

If you want to start a new conversation, all you have to do is change the `thread_id` used.

In [87]:
config = {"configurable": {"thread_id": "xyz123"}}

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="What's my name?")]},
    config
):
    rprint(chunk)
    print("-" * 100)

{
    'agent': {
        'messages': [
            AIMessage(
                content="I'm sorry, but I don't have access to personal information like your name. How can I 
assist you today?",
                response_metadata={
                    'token_usage': {'completion_tokens': 25, 'prompt_tokens': 86, 'total_tokens': 111},
                    'model_name': 'gpt-3.5-turbo',
                    'system_fingerprint': None,
                    'finish_reason': 'stop',
                    'logprobs': None
                },
                id='run-8e5a6a4d-2f21-47be-bed4-a9375fad6b8f-0',
                usage_metadata={'input_tokens': 86, 'output_tokens': 25, 'total_tokens': 111}
            )
        ]
    }
}

----------------------------------------------------------------------------------------------------
